In [30]:
from iFinDPy import *
from datetime import datetime
import pandas as pd
import numpy as np

In [31]:
def thslogindemo():
    # 输入用户的帐号和密码
    thsLogin = THS_iFinDLogin("gdgy072","06a3c6")
    print(thsLogin)
    if thsLogin != 0:
        print('登录失败')
    else:
        print('登录成功')
thslogindemo()

-201
登录失败


In [32]:
days = 365 * 10

In [33]:
#经济数据库(EDB)-中国宏观数据-中国综合PMI:产出指数;社会融资规模增量:当月值;金融机构:中长期贷款余额;进出口总值(人民币计价):当月环比等-iFinD数据接口
EDB_data = THS_EDB('M004386385;M004891015;M003146165;M00410905;M002043803;M002043804;M002043811;M002043807;M001625221;M001625223;M002826785;M001625518;M002822189;M001622755;M004392434;M004891011;M005959895;M005959894;M002842661;M001620723;G002600774;S004094483;M006784027;M006784025','',(datetime.now()-timedelta(days=days)).strftime("%Y-%m-%d"),datetime.now().strftime("%Y-%m-%d")).data
#日期序列-指数-收盘价;涨跌幅;
SCH_data = THS_HQ('SCH003012.SCH,SCH003022.SCH','close,changeRatio','',(datetime.now()-timedelta(days=days)).strftime("%Y-%m-%d"),datetime.now().strftime("%Y-%m-%d")).data

In [34]:
with pd.ExcelWriter('original_data.xlsx', engine='xlsxwriter') as writer:
    EDB_data.to_excel(writer, sheet_name='EDB_data', index=False)
    SCH_data.to_excel(writer, sheet_name='SCH_data', index=False)

In [35]:
def up_days(close_data):
    up_days = [0]
    current_up_count = 0
    for i in range(1, len(close_data)):
        if close_data[i] > close_data[i - 1]:
            current_up_count += 1
        else:
            current_up_count = 0
        up_days.append(current_up_count)
    return up_days

def slowKD(close, n=14):
    data = pd.DataFrame({'Close': close})
    data['Lowest_Low'] = data['Close'].rolling(n).min()
    data['Highest_High'] = data['Close'].rolling(n).max()
    data['%K'] = (data['Close'] - data['Lowest_Low']) / (data['Highest_High'] - data['Lowest_Low']) * 100

    slow_k_period = 3
    data['Slow_%K'] = data['%K'].rolling(slow_k_period).mean()

    slow_d_period = 3
    data['Slow_%D'] = data['Slow_%K'].rolling(slow_d_period).mean()
    return data['Slow_%K'], data['Slow_%D']

def calculate_SMA(data, window_size):
    sma = [] 

    for i in range(window_size - 1, len(data)):
        average = sum(data[i - window_size + 1:i + 1]) / window_size
        sma.append(average)

    return sma

def breakout_ma(data, window_size=5):
    sma = calculate_SMA(data, window_size)
    breakouts = [None] * (window_size - 1)

    for i in range(window_size - 1, len(data)):
        if data[i] > sma[i - window_size + 1]:
            breakouts.append(1)
        else:
            breakouts.append(0)

    return breakouts

def breakdown_ma(close):
    return [-x+1 if x!=None else None for x in breakout_ma(close) ]

def std_3m(changeRatio, window_size = 65):
    return changeRatio.rolling(window=window_size).std()

def std_6m(changeRatio, window_size = 130):
    return changeRatio.rolling(window=window_size).std()

def BOLL(close, n=20):
    return close.rolling(window=n).mean() + 2 * close.rolling(window=n).std(), close.rolling(window=n).mean() - 2 * close.rolling(window=n).std()

In [36]:
pivoted_data = []
statistics_list = ['房地产开发投资:累计值','汽车:产量:当月值','M2(货币和准货币):期末值','M1(货币):期末值','社会消费品零售总额:当月值','制造业PMI:生产','制造业PMI:新订单','制造业PMI:产成品库存','制造业PMI:原材料库存','规模以上工业增加值:季调:环比','CPI:环比','PPI:环比','金融机构:中长期贷款余额','中国综合PMI:产出指数','规模以上工业企业:利润总额:当月值','社会融资规模增量:政府债券:当月值','社会融资规模增量:当月值']
statistics_dic = dict()
for group_name, group in EDB_data.groupby('index_name'):
    pivoted_df = group.pivot(index='time', columns='index_name', values='value')
    if group_name in statistics_list:
        statistics_dic[group_name] = pivoted_df.set_index(pd.to_datetime(pivoted_df.index).strftime('%Y-%m'), inplace=False)[group_name].to_dict()
    pivoted_data.append(pivoted_df)

merged_df_EDB = pd.concat(pivoted_data, axis=1, join='outer').sort_index()
merged_df_EDB.index = pd.to_datetime(merged_df_EDB.index)

for index, row in merged_df_EDB.iterrows():
    for c in statistics_list:
        if index.day <= 15:
            merged_df_EDB.at[index, c] = statistics_dic[c].get((pd.to_datetime(index.strftime('%Y-%m'), format='%Y-%m') - pd.DateOffset(months=2)).strftime('%Y-%m'), None)
        else:
            merged_df_EDB.at[index, c] = statistics_dic[c].get((pd.to_datetime(index.strftime('%Y-%m'), format='%Y-%m') - pd.DateOffset(months=1)).strftime('%Y-%m'), None)

merged_df_EDB.fillna(method='ffill', inplace=True)
merged_df_EDB['year'] = merged_df_EDB.index.year
merged_df_EDB['month'] = merged_df_EDB.index.month
merged_df_EDB['day'] = merged_df_EDB.index.day

pivoted_data = []

for group_name, group in SCH_data.groupby('thscode'):
    group['up_days'] = up_days(group['close'].to_list())
    group['slowK'] = slowKD(group['close'].to_list())[0]
    group['slowD'] = slowKD(group['close'].to_list())[1]
    group['breakout_ma'] = breakout_ma(group['close'].to_list())
    group['breakdown_ma'] = breakdown_ma(group['close'].to_list())
    group['std_3m'] = std_3m(group['changeRatio'])
    group['std_6m'] = std_6m(group['changeRatio'])
    group['BOLL_hband'] = BOLL(group['close'])[0]
    group['BOLL_lband'] = BOLL(group['close'])[1]
    pivoted_data.append(group.set_index('time').drop(['thscode'], axis=1).add_prefix(group_name+'_'))

merged_df_SCH = pd.concat(pivoted_data, axis=1, join='outer').sort_index()
merged_df_SCH.fillna(method='ffill', inplace=True)
merged_df_SCH.index = pd.to_datetime(merged_df_SCH.index)

merged_df_whole = pd.merge(merged_df_EDB, merged_df_SCH, how='outer', on='time').sort_index()
merged_df_whole.fillna(method='ffill', inplace=True)

merged_df_SCH['year'] = merged_df_SCH.index.year
merged_df_SCH['month'] = merged_df_SCH.index.month
merged_df_SCH['day'] = merged_df_SCH.index.day

with pd.ExcelWriter('preprosed_data.xlsx', engine='xlsxwriter') as writer:
    merged_df_EDB.to_excel(writer, sheet_name='EDB_data', index=False)
    merged_df_SCH.to_excel(writer, sheet_name='SCH_data', index=False)

merged_df_whole.to_excel('whole_data.xlsx',index=False)

In [37]:
merged_df_whole

,CPI:环比,M1(货币):期末值,M2(货币和准货币):期末值,PPI:环比,中国综合PMI:产出指数,制造业PMI:产成品库存,制造业PMI:原材料库存,制造业PMI:新订单,制造业PMI:生产,南华期货:工业品指数,...,SCH003022.SCH_changeRatio,SCH003022.SCH_up_days,SCH003022.SCH_slowK,SCH003022.SCH_slowD,SCH003022.SCH_breakout_ma,SCH003022.SCH_breakdown_ma,SCH003022.SCH_std_3m,SCH003022.SCH_std_6m,SCH003022.SCH_BOLL_hband,SCH003022.SCH_BOLL_lband
time,,,,,,,,,,,,,,,,,,,,,
2013-11-07,None,None,None,None,None,None,None,None,None,1857.859,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-08,None,None,None,None,None,None,None,None,None,1849.675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-11,None,None,None,None,None,None,None,None,None,1852.042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-12,None,None,None,None,None,None,None,None,None,1850.212,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-13,None,None,None,None,None,None,None,None,None,1841.625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-30,0.2,678443.65,2896659.11,0.4,52.0,46.7,48.5,50.5,52.7,4030.23,...,0.069085,4.0,NaN,NaN,1.0,0.0,0.033064,0.028924,149.082100,148.785990
2023-10-31,0.2,678443.65,2896659.11,0.4,52.0,46.7,48.5,50.5,52.7,4000.42,...,0.012144,5.0,NaN,NaN,1.0,0.0,0.032828,0.028915,149.079819,148.820811
2023-11-01,0.2,678443.65,2896659.11,0.4,52.0,46.7,48.5,50.5,52.7,3991.65,...,0.024754,6.0,NaN,NaN,1.0,0.0,0.032776,0.028922,149.078573,148.857187


In [38]:
df = merged_df_whole

half_year_ago = pd.Timestamp('now') - pd.Timedelta(days=180)

df_test = df[df.index >= half_year_ago]
df_train = df[df.index < half_year_ago]

In [39]:
df_test

,CPI:环比,M1(货币):期末值,M2(货币和准货币):期末值,PPI:环比,中国综合PMI:产出指数,制造业PMI:产成品库存,制造业PMI:原材料库存,制造业PMI:新订单,制造业PMI:生产,南华期货:工业品指数,...,SCH003022.SCH_changeRatio,SCH003022.SCH_up_days,SCH003022.SCH_slowK,SCH003022.SCH_slowD,SCH003022.SCH_breakout_ma,SCH003022.SCH_breakdown_ma,SCH003022.SCH_std_3m,SCH003022.SCH_std_6m,SCH003022.SCH_BOLL_hband,SCH003022.SCH_BOLL_lband
time,,,,,,,,,,,,,,,,,,,,,
2023-05-10,-0.3,678059.63,2814566.31,0.0,57.0,49.5,48.3,53.6,54.6,3578.03,...,0.034284,8.0,NaN,NaN,1.0,0.0,0.025699,0.057984,146.838153,145.784567
2023-05-11,-0.3,678059.63,2814566.31,0.0,57.0,49.5,48.3,53.6,54.6,3511.71,...,0.041904,9.0,NaN,NaN,1.0,0.0,0.025755,0.058052,146.900782,145.809758
2023-05-12,-0.3,678059.63,2814566.31,0.0,57.0,49.5,48.3,53.6,54.6,3478.41,...,0.029695,10.0,NaN,NaN,1.0,0.0,0.025755,0.058046,146.959887,145.841073
2023-05-15,-0.3,678059.63,2814566.31,0.0,57.0,49.5,48.3,53.6,54.6,3520.15,...,0.038537,11.0,NaN,NaN,1.0,0.0,0.025788,0.058092,147.019300,145.874910
2023-05-16,-0.1,669761.55,2808469.34,-0.5,54.4,49.4,47.9,48.8,50.2,3509.4,...,-0.002859,0.0,NaN,NaN,1.0,0.0,0.026052,0.058092,147.070240,145.909620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-30,0.2,678443.65,2896659.11,0.4,52.0,46.7,48.5,50.5,52.7,4030.23,...,0.069085,4.0,NaN,NaN,1.0,0.0,0.033064,0.028924,149.082100,148.785990
2023-10-31,0.2,678443.65,2896659.11,0.4,52.0,46.7,48.5,50.5,52.7,4000.42,...,0.012144,5.0,NaN,NaN,1.0,0.0,0.032828,0.028915,149.079819,148.820811
2023-11-01,0.2,678443.65,2896659.11,0.4,52.0,46.7,48.5,50.5,52.7,3991.65,...,0.024754,6.0,NaN,NaN,1.0,0.0,0.032776,0.028922,149.078573,148.857187


In [40]:
df_train

,CPI:环比,M1(货币):期末值,M2(货币和准货币):期末值,PPI:环比,中国综合PMI:产出指数,制造业PMI:产成品库存,制造业PMI:原材料库存,制造业PMI:新订单,制造业PMI:生产,南华期货:工业品指数,...,SCH003022.SCH_changeRatio,SCH003022.SCH_up_days,SCH003022.SCH_slowK,SCH003022.SCH_slowD,SCH003022.SCH_breakout_ma,SCH003022.SCH_breakdown_ma,SCH003022.SCH_std_3m,SCH003022.SCH_std_6m,SCH003022.SCH_BOLL_hband,SCH003022.SCH_BOLL_lband
time,,,,,,,,,,,,,,,,,,,,,
2013-11-07,None,None,None,None,None,None,None,None,None,1857.859,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-08,None,None,None,None,None,None,None,None,None,1849.675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-11,None,None,None,None,None,None,None,None,None,1852.042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-12,None,None,None,None,None,None,None,None,None,1850.212,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-13,None,None,None,None,None,None,None,None,None,1841.625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-04,-0.3,678059.63,2814566.31,0.0,57.0,49.5,48.3,53.6,54.6,3528.37,...,0.095154,3.0,NaN,NaN,1.0,0.0,0.026811,0.057900,146.534545,145.652685
2023-05-05,-0.3,678059.63,2814566.31,0.0,57.0,49.5,48.3,53.6,54.6,3530.04,...,0.044563,4.0,NaN,NaN,1.0,0.0,0.026712,0.057915,146.589151,145.690749
2023-05-06,-0.3,678059.63,2814566.31,0.0,57.0,49.5,48.3,53.6,54.6,3530.04,...,0.037449,5.0,NaN,NaN,1.0,0.0,0.026345,0.057965,146.655994,145.710436
